In [1]:
import gadata_funcs,pdb,re
import pandas as pd
import numpy as np
import pylab as pl
from gadata_funcs import date2days,gadata_date2days,salesdata_date2days,getTrainingData
from importlib import reload
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.linear_model import LogisticRegression
from sys import stdout
from xgboost import XGBClassifier

%matplotlib notebook

In [2]:
# Conducting the experiments
################################

# Parameters
nAugment=1
pAugment=.85
labelWindow=20
featureWindow=30
labelThreshold=[150]
featureList=['hits','maxPurchase']

# Loading data
xdf,ydf=getTrainingData(labelThreshold,labelWindow=labelWindow,featureWindow=featureWindow)
x=xdf[featureList].values #,'maxPurchase']].values 
y=ydf.values.reshape((-1,))

scores=[]
importances=[]
print("Cross validation round: ")
for count in range(20):
    stdout.write("#"+str(count)+",")
    
    #rf=GradientBoostingClassifier(n_estimators=300)
    #rf=XGBClassifier(n_estimators=250,max_depth=6)
    rf=RandomForestClassifier(n_estimators=350,max_depth=6)
    
    xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=.05)
    
    # Augmenting minority class
    for count2 in range(nAugment):
        minoritySamp=np.logical_and(ytrain==1,np.random.rand(len(ytrain))<pAugment)
        xtrain=np.concatenate((xtrain,xtrain[minoritySamp]),axis=0)
        ytrain=np.concatenate((ytrain,np.ones(np.sum(minoritySamp))),axis=0)
    
    rf.fit(xtrain,ytrain)
    importances.append(rf.feature_importances_)
    scores.append([accuracy_score(ytest,rf.predict(xtest)),precision_score(ytest,rf.predict(xtest)),recall_score(ytest,rf.predict(xtest))])

print()
scores=pd.DataFrame(scores,columns=['Accuracy','Precision','Recall'])
print(np.mean(scores))

Cross validation round: 
#0,#1,#2,#3,#4,#5,#6,#7,#8,#9,#10,#11,#12,#13,#14,#15,#16,#17,#18,#19,
Accuracy     0.727645
Precision    0.585002
Recall       0.678907
dtype: float64
